---<br>
jupyter:<br>
  jupytext:<br>
    text_representation:<br>
      extension: .py<br>
      format_name: light<br>
      format_version: '1.5'<br>
      jupytext_version: 1.14.4<br>
  kernelspec:<br>
    display_name: Python 3 (ipykernel)<br>
    language: python<br>
    name: python3<br>
---

# Cylindrical Stokes (Cartesian formulation)<br>
<br>
Let the mesh deform to create a free surface. If we iterate on this, then it is almost exactly the same as the free-slip boundary condition (though there are potentially instabilities here).<br>
<br>
The problem has a constant velocity nullspace in x,y. We eliminate this by fixing the central node in this example, but it does introduce a perturbation to the flow near the centre which is not always stagnant.

+

In [ ]:
import petsc4py
from petsc4py import PETSc

In [ ]:
import underworld3 as uw
from underworld3.systems import Stokes
from underworld3 import function

In [ ]:
import numpy as np
import sympy

In [ ]:
free_slip_upper = True

In [ ]:
import os

In [ ]:
os.environ["UW_TIMING_ENABLE"] = "1"

Define the problem size<br>
     1 - ultra low res for automatic checking<br>
     2 - low res problem to play with this notebook<br>
     3 - medium resolution (be prepared to wait)<br>
     4 - highest resolution (benchmark case from Spiegelman et al)

In [ ]:
problem_size = 2

For testing and automatic generation of notebook output,<br>
over-ride the problem size if the UW_TESTING_LEVEL is set

In [ ]:
uw_testing_level = os.environ.get("UW_TESTING_LEVEL")
if uw_testing_level:
    try:
        problem_size = int(uw_testing_level)
    except ValueError:
        # Accept the default value
        pass

In [ ]:
r_o = 1.0
r_i = 0.5

In [ ]:
if problem_size <= 1:
    res = 0.2
elif problem_size == 2:
    res = 0.1
elif problem_size == 3:
    res = 0.05
elif problem_size == 4:
    res = 0.025
elif problem_size == 5:
    res = 0.01
elif problem_size >= 6:
    res = 0.005
# -

In [ ]:
meshball = uw.meshing.Annulus(radiusOuter=r_o, radiusInner=r_i, cellSize=res)

In [ ]:
meshball.dm.view()

+<br>
Test that the second one is skipped

In [ ]:
v_soln = uw.discretisation.MeshVariable(r"\mathbf{u}", meshball, 2, degree=2)
p_soln = uw.discretisation.MeshVariable(r"p", meshball, 1, degree=1, continuous=True)
p_cont = uw.discretisation.MeshVariable(r"p_c", meshball, 1, degree=1, continuous=True)
t_soln = uw.discretisation.MeshVariable(r"\Delta T", meshball, 1, degree=3)
maskr = uw.discretisation.MeshVariable("r", meshball, 1, degree=1)

+<br>
Create a density structure / buoyancy force<br>
gravity will vary linearly from zero at the centre<br>
of the sphere to (say) 1 at the surface

In [ ]:
import sympy

In [ ]:
radius_fn = meshball.CoordinateSystem.xR[0]
radius_fn = maskr.sym[0]
unit_rvec = meshball.CoordinateSystem.unit_e_0
gravity_fn = 1  # radius_fn / r_o

Some useful coordinate stuff

In [ ]:
x, y = meshball.CoordinateSystem.X
r, th = meshball.CoordinateSystem.xR

In [ ]:
Rayleigh = 1.0e5

In [ ]:
hw = 2000.0 / res
surface_fn = sympy.exp(-((radius_fn - r_o) ** 2) * hw)
base_fn = sympy.exp(-((radius_fn - r_i) ** 2) * hw)

+<br>
Create Stokes object

In [ ]:
stokes = Stokes(
    meshball, velocityField=v_soln, pressureField=p_soln, solver_name="stokes"
)

In [ ]:
stokes.constitutive_model = uw.systems.constitutive_models.ViscousFlowModel(
    meshball.dim
)
stokes.constitutive_model.Parameters.viscosity = 1

There is a null space if there are no fixed bcs, so we'll do this:

In [ ]:
if not free_slip_upper:
    stokes.add_dirichlet_bc((0.0, 0.0), "Upper", (0, 1))

In [ ]:
stokes.add_dirichlet_bc((0.0, 0.0), "Lower", (0, 1))

-

In [ ]:
pressure_solver = uw.systems.Projection(meshball, p_cont)
pressure_solver.uw_function = p_soln.sym[0]
pressure_solver.smoothing = 1.0e-3

t_init = 10.0 * sympy.exp(-5.0 * (x**2 + (y - 0.5) ** 2))

In [ ]:
t_init = sympy.cos(3 * th)

+<br>
Write density into a variable for saving

In [ ]:
with meshball.access(t_soln):
    t_soln.data[:, 0] = uw.function.evaluate(
        t_init, coords=t_soln.coords, coord_sys=meshball.N
    )
    print(t_soln.data.min(), t_soln.data.max())

In [ ]:
with meshball.access(maskr):
    maskr.data[:, 0] = uw.function.evaluate(
        r, coords=maskr.coords, coord_sys=meshball.N
    )

+

In [ ]:
I = uw.maths.Integral(meshball, surface_fn)
s_norm = I.evaluate()
print(s_norm)

In [ ]:
I.fn = base_fn
b_norm = I.evaluate()
print(b_norm)
# +

In [ ]:
buoyancy_force = Rayleigh * gravity_fn * t_init
if free_slip_upper:
    buoyancy_force -= 1.0e6 * v_soln.sym.dot(unit_rvec) * surface_fn / s_norm
    buoyancy_force -= 1.0e6 * v_soln.sym.dot(unit_rvec) * base_fn / b_norm

In [ ]:
stokes.bodyforce = unit_rvec * buoyancy_force

This may help the solvers - penalty in the preconditioner

In [ ]:
stokes.saddle_preconditioner = 1.0

In [ ]:
stokes.petsc_options["ksp_monitor"] = None
stokes.tolerance = 1.0e-4

-

In [ ]:
stokes.petsc_options.getAll()

+

In [ ]:
from underworld3 import timing

In [ ]:
timing.reset()
timing.start()
stokes.solve(zero_init_guess=True)
timing.print_table()
print("", flush=True)
# -

In [ ]:
stokes._u_f0[0]

+<br>
Pressure at mesh nodes

In [ ]:
pressure_solver.solve()

+<br>
check the mesh if in a notebook / serial

In [ ]:
if uw.mpi.size == 1:
    import numpy as np
    import pyvista as pv
    import vtk
    pv.global_theme.background = "white"
    pv.global_theme.window_size = [750, 600]
    pv.global_theme.antialiasing = True
    pv.global_theme.jupyter_backend = "panel"
    pv.global_theme.smooth_shading = True
    meshball.vtk("tmp_ball.vtk")
    pvmesh = pv.read("tmp_ball.vtk")
    with meshball.access():
        pvmesh.point_data["V"] = uw.function.evaluate(
            v_soln.sym.dot(v_soln.sym), meshball.data
        )
        pvmesh.point_data["P"] = uw.function.evaluate(p_cont.sym[0], meshball.data)
        pvmesh.point_data["T"] = uw.function.evaluate(
            t_init, meshball.data, coord_sys=meshball.N
        )
    with meshball.access():
        usol = stokes.u.data
    arrow_loc = np.zeros((stokes.u.coords.shape[0], 3))
    arrow_loc[:, 0:2] = stokes.u.coords[...]
    arrow_length = np.zeros((stokes.u.coords.shape[0], 3))
    arrow_length[:, 0:2] = usol[...]
    # -
    pl = pv.Plotter(window_size=(750, 750))

    # pl.add_mesh(pvmesh,'Black', 'wireframe')
    pl.add_mesh(
        pvmesh,
        cmap="coolwarm",
        edge_color="Grey",
        scalars="T",
        show_edges=True,
        use_transparency=False,
        opacity=0.75,
    )
    pl.add_arrows(arrow_loc, arrow_length, mag=0.0001)
    pl.show(cpos="xy")
# -
usol_rms = np.sqrt(usol[:, 0] ** 2 + usol[:, 1] ** 2).mean()
usol_rms